In [3]:
# stanza.download("fr")
# stanza.download("nl")

In [2]:
from collections import Counter
import pandas as pd
import stanza
import re

In [24]:
en_gt = pd.read_csv('../ACTER/en/htfl/annotations/htfl_en_terms.ann', sep='	', engine='python',header=None)
en_preds = pd.read_csv('../results/en/ann/lem_roberta_ann_predictions.csv')
preds = list(en_preds['predictions'])
gts = list(en_gt[0])

In [25]:
def count_freq(preds, gts):
    preds_len = [len(x.split(' ')) for x in preds]
    gts_len = [len(x.split(' ')) for x in gts]
    print(Counter(preds_len))
    print(Counter(gts_len))

In [26]:
preds_len = [len(x.split(' ')) for x in preds]
gts_len = [len(x.split(' ')) for x in gts]
print(Counter(preds_len))
print(Counter(gts_len))

Counter({1: 638, 2: 56, 3: 6})
Counter({1: 1064, 2: 766, 3: 357, 4: 120, 5: 31, 6: 18, 7: 4, 8: 1})


In [27]:
max(gts, key=len)

'heart failure with preserved left ventricular ejection fraction'

In [29]:
# gts[gts.str.split(' ').len() == 3]

In [19]:
# P = TP/(TP+FP)
# R =  TP/(TP+FN)
def evaluation_metrics(pred, gt):
    TP = len(set(pred) & set(gt)) 
    FP = len(set(pred)-set(gt))
    FN = len(set(gt)-set(pred))
    print(TP, FP, FN)
    precision = round((TP/(TP+FP))*100, 2)
    recall = round((TP/(TP+FN))*100,2)
    f1_score = round((2 * precision * recall) / (precision + recall),2)
    return precision, recall, f1_score

precision, recall, f1_score = evaluation_metrics(preds, gts)
print(len(set(preds)))
print(len(set(gts)))
precision, recall, f1_score

336 372 2025
708
2361


(47.46, 14.23, 21.9)

Stanza

In [20]:
nlp = stanza.Pipeline(lang='en')# processors='tokenize,mwt,pos,lemma'

2021-04-09 16:12:35 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-04-09 16:12:35 INFO: Use device: cpu
2021-04-09 16:12:35 INFO: Loading: tokenize
2021-04-09 16:12:35 INFO: Loading: pos
2021-04-09 16:12:36 INFO: Loading: lemma
2021-04-09 16:12:36 INFO: Loading: depparse
2021-04-09 16:12:38 INFO: Loading: sentiment
2021-04-09 16:12:39 INFO: Loading: ner
2021-04-09 16:12:40 INFO: Done loading processors!


In [21]:
def lemma(li):
    new_list = []
    for t in li:
        doc = nlp(str(t))
        doc1 = ' '.join([word.lemma for sent in doc.sentences for word in sent.words])
        doc1 = re.sub('-',' ',doc1)
        doc1 = re.sub(' +', ' ',doc1)
        new_list.append(doc1)
    new_list = [s for s in new_list if len(s) >= 2]
    return new_list

In [22]:
preds_lem = lemma(preds)
gts_lem = lemma(gts)

print('After lemmatization')
print(len(set(preds_lem)))
print(len(set((gts_lem))))
print(evaluation_metrics(list(set(preds_lem)), list(set(gts_lem))))

After lemmatization
676
2149
320 356 1829
(47.34, 14.89, 22.65)


In [23]:
set(preds_lem)

{'absolute',
 'absorptiometry',
 'absorption',
 'acp',
 'active',
 'activity',
 'acute',
 'adenosine',
 'adipokine',
 'administrative',
 'admission',
 'adrenergic',
 'adrenoceptor',
 'aerobic',
 'aerobic capacity',
 'aetiology',
 'aim',
 'alcohol',
 'ambulatory',
 'ami',
 'amyloidosis',
 'amyloidotic',
 'anaerobic',
 'anastomosis',
 'anemia',
 'angiogenesis',
 'angiography',
 'angioplasty',
 'angiotensin',
 'ankle',
 'annular',
 'annular plane',
 'annuloplasty',
 'anterior',
 'antirheumatic',
 'aortic',
 'apoptosis',
 'apoptotic',
 'appendicular',
 'arm',
 'armamentarium',
 'arrhythmia',
 'arterial',
 'arthritis',
 'assist',
 'atherosclerosis',
 'atp',
 'atrioventricular',
 'automatic',
 'axis',
 'background',
 'balance',
 'baroreflex',
 'baroreflex activation',
 'bat',
 'battery',
 'beat',
 'benign tumor',
 'beta',
 'biomarker',
 'bleed',
 'blockade',
 'blocker',
 'blood',
 'bmi',
 'bolus',
 'box',
 'brachial',
 'brain',
 'branch',
 'breast',
 'breath',
 'breathlessness',
 'bridge',
 

In [14]:
set(gts_lem)

{'p value',
 'etiologic',
 'interleukin receptor',
 'organ',
 'cardiogenic',
 'tricuspid',
 'ckd',
 'ser77',
 'myocardial remodelling',
 'endothelial no synthase activation',
 'fat free mass',
 'nesiritide',
 'end diastolic pressure',
 'invasive',
 'gene induction',
 'monomeric g protein',
 'beta adrenergic receptor',
 'retn',
 'systolic leave ventricular dysfunction',
 'anti neoplastic agent',
 'correlation coefficient',
 'myocardial oxygen consumption',
 'point of care',
 'ehrlich carcinoma',
 'peak oxygen uptake',
 'chronic pulmonary arterial hypertension',
 'diabete mellitus',
 'ste',
 "e / e '",
 'parasympathetic tone',
 'caspase 3',
 'fluid overload',
 'anthracycline induce cardiotoxicity',
 'perfusion',
 'paf',
 'mammalian enable',
 'ser96',
 'congestive hf',
 'mitochondrial respiration',
 'perk',
 'remodelling',
 'cardiomyiopathy',
 'blood pressure',
 'kaplan meier analysis',
 'antibody',
 'bet',
 'rad variant',
 'chronic hfpef',
 'anemia',
 'adipose tissue',
 'ies',
 'gly7',
 

NLTK

In [ ]:
# from nltk.stem import PorterStemmer, WordNetLemmatizer
# port = PorterStemmer()
# wnl = WordNetLemmatizer()

# preds_stem = [port.stem(i) for i in preds]
# gts_stem = [port.stem(i) for i in gts]

# print('After stemming')
# print(len(set(preds_stem)))
# print(len(set(gts_stem)))
# print(evaluation_metrics(list(set(preds_stem)), list(set(gts_stem))))

# preds_lem = [wnl.lemmatize(i) for i in preds]
# gts_lem = [wnl.lemmatize(i) for i in gts]

# print('After lemmatization')
# print(len(set(preds_lem)))
# print(len(set((gts_lem))))
# print(evaluation_metrics(list(set(preds_lem)), list(set(gts_lem))))

# preds_com = [port.stem(wnl.lemmatize(i)) for i in preds]
# gts_com = [port.stem(wnl.lemmatize(i)) for i in gts]

# print('After combined')
# print(len(set(preds_com)))
# print(len(set((gts_com))))
# evaluation_metrics(list(set(preds_com)), list(set(gts_com)))

# preds_ =[x for x in preds if len(x) >= 2]
# gts_ =[x for x in gts if len(x) >= 2]
# print(len(set(preds_)))
# print(len(gts))
# evaluation_metrics(set(preds_), gts)